In [ ]:
# df1 = pd.read_csv('resume_data_improved.csv')
# df2 = pd.read_csv('resume_data_developer.csv')
# pattern = "resume_data"
# files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.startswith(pattern)]
# df_full = [pd.read_csv(csv) for csv in files]
# df = pd.concat(df_full, axis=0, ignore_index=True).drop_duplicates()
# df.shape

In [ ]:
df.head()

In [ ]:
# df = pd.concat([df1, df2], axis=0).drop_duplicates()

In [ ]:
df.isnull().sum() / df.shape[0] * 100

In [ ]:
df.info()

In [ ]:
df = df.drop(['Availability Date', 'Date Posted', 'Unnamed: 0', 'Reference'], axis=1)

In [ ]:
df.shape

In [ ]:
df.nunique()

In [ ]:
df[['Willing to Relocate', 'U.S. Work Authorization', 'Willing to Travel']].value_counts()

In [ ]:
# FEATURE PREPROCESSING
import numpy as np, re
df['Willing to Relocate'] = df['Willing to Relocate'].map({'No': 'no', 'Yes': 'yes'})
df.loc[df['U.S. Work Authorization'] == 'Other', 'U.S. Work Authorization'] = np.nan

In [ ]:
def myfunc(a):

    if a is np.nan:
        return np.nan
    res = re.search(r'([0-9]+-)*[0-9]+%$', a)
    if res and res.group() in ['50-75%', '25%', '25-50%', '75%']:
        return res.group()[:-1]
    else:
        return a

df['Willing to Travel'] = df['Willing to Travel'].apply(myfunc)
df['Willing to Travel'] = df['Willing to Travel'].map({'50-75': 0.625, 'No': 0, '25': 0.25, '25-50': 0.375})

In [ ]:
df.info()

In [ ]:
df['Desired Industry'].value_counts().sort_values(ascending=False)

In [ ]:
df['Role'].sort_values().unique()

# START FROM HERE

In [1]:
import re, numpy as np
import os, pandas as pd
directory = os.path.join(os.getcwd())
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and re.search(r'category_[0-9]+', f)]

In [2]:
df2 = pd.concat([pd.read_csv(csv) for csv in files], axis=0, ignore_index=True).drop_duplicates()

In [3]:
df2['Desired Industry'].value_counts()

Engineering                       1500
Information Technology            1500
Accounting/Bookkeeping            1500
Business/Management               1500
Finance/Investment                1276
Computer Software/Programming     1057
Banking/Mortgage                  1007
Graphics Design/Commercial Art     604
CAD/Drafting                       556
Automotive/Mechanic                483
Computer Hardware                  330
Quality Control                    289
Public Relations                   227
Name: Desired Industry, dtype: int64

In [4]:
df2 = df2.drop(['Availability Date', 'Date Posted', 'Unnamed: 0', 'Reference', 'Affiliations', 'Additional Information'], axis=1)

In [5]:
df2.shape

(11829, 15)

In [6]:

attr = ['Skills', 'Objective', 'Education', 'Experience']
for col in attr:
    df2[col] = df2[col].apply(lambda x: str(x).encode('ascii', errors='ignore').decode().strip())
    df2[col] = df2[col].replace(r'^[-=]+[-=]$', np.nan ,  regex=True)
    df2[col] = df2[col].replace(['nan', 'non'], np.nan)
    df2[col] = df2[col].str.replace('\t', '')
    df2[col] = df2[col].str.replace('\n', '.')

In [7]:
df2.columns

Index(['Role', 'Desired Industry', 'SpiderID', 'Desired Job Location',
       'Type of Position', 'Desired Wage', 'U.S. Work Authorization',
       'Job Level', 'Willing to Travel', 'Highest Degree Attained',
       'Willing to Relocate', 'Objective', 'Experience', 'Education',
       'Skills'],
      dtype='object')

In [8]:
df2['Desired Industry'].value_counts()

Engineering                       1500
Information Technology            1500
Accounting/Bookkeeping            1500
Business/Management               1500
Finance/Investment                1276
Computer Software/Programming     1057
Banking/Mortgage                  1007
Graphics Design/Commercial Art     604
CAD/Drafting                       556
Automotive/Mechanic                483
Computer Hardware                  330
Quality Control                    289
Public Relations                   227
Name: Desired Industry, dtype: int64

In [9]:
df2.Skills

0        Preventive Maintenance.Process Improvement.Cha...
1                                                     java
2        Expert in reviewing mechanical site design dra...
3                   Mechanichal Engineering + Mechatronics
4        Leadership Role: HKIE Student Chapters Committ...
                               ...                        
11824                                                  NaN
11825                                                  NaN
11826                                                  NaN
11827                                                  NaN
11828                                                  NaN
Name: Skills, Length: 11829, dtype: object

In [10]:
df2.shape

(11829, 15)

In [11]:
df2 = df2.fillna('')

In [80]:
# df2.to_csv('data.csv')

In [81]:
df['Skills'] + ' '+ df['Objective']

NameError: name 'df' is not defined

In [12]:
df2.isnull().sum()

Role                       0
Desired Industry           0
SpiderID                   0
Desired Job Location       0
Type of Position           0
Desired Wage               0
U.S. Work Authorization    0
Job Level                  0
Willing to Travel          0
Highest Degree Attained    0
Willing to Relocate        0
Objective                  0
Experience                 0
Education                  0
Skills                     0
dtype: int64

# **NLP PART**

In [13]:
import spacy

In [14]:
nlp = spacy.load('en_core_web_sm')

In [15]:
df_copy = df2.copy()

In [18]:
def preprocess_doc(doc):
    # Tokenize the document
    # tokens = [token.text for token in doc]

    # Remove stop words
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    # Join the tokens back into a single string
    preprocessed_doc = ' '.join(tokens)

    return preprocessed_doc

In [20]:
for col in ['Skills', 'Objective', 'Education', 'Experience']:
    df_copy[col] = df_copy[col].apply(lambda x: preprocess_doc(nlp(x)))

In [21]:
df_copy[['Skills', 'Objective', 'Education', 'Experience']]

,Skills,Objective,Education,Experience
0,Preventive Maintenance Process Improvement Cha...,skilled carry preventative corrective general ...,Bachelor Arts Geosciences University North C...,Metro Ports Wilmington NC.Maintenance Manager ...
1,java,goal orient hardworke engineer graduate keen i...,Philippines Licensed Mechanical Engineer 15 ye...,versatile technically astute field piping qc t...
2,expert review mechanical site design drawing r...,hand analytical solution focus professional ex...,Bachelor Technology Mechanichal Engineering+ A...,Technical Management Trainee Eltex Appliances ...
3,Mechanichal Engineering + mechatronic,secure challenging role expand knowledge skill...,Bachelor Engineering Mechanical Engineering Ho...,hand analytical solution focus professional ex...
4,Leadership Role HKIE Student Chapters Committe...,TARGET Jobspider Senior Mems Process Engineer ...,hand analytical solution focus professional ex...,Gururaj Engineers automation Electrical Engine...
...,...,...,...,...
11824,,,,
11825,,,,
11826,,,,
11827,,,,


In [25]:
df_copy.to_csv('data.csv')

In [65]:
df_copy

,Role,Desired Industry,SpiderID,Desired Job Location,Type of Position,Desired Wage,U.S. Work Authorization,Job Level,Willing to Travel,Highest Degree Attained,Willing to Relocate,Objective,Experience,Education,Skills
0,Director of Maintenance Operations Resume,Engineering,83950,"Wilmington, North Carolina",Full-Time Permanent,115000,Yes,"Management (Manager, Director)","Yes, Less Than 25%",Bachelors,Yes,,,,
1,java developer Resume,Engineering,83927,"new york, Delaware",Contractor,,Yes,"Management (Manager, Director)","Yes, More Than 75%",Bachelors,Yes,,,,
2,Mechanical Engineer Resume,Engineering,83916,"Surrey, New Brunswick",Full-Time Permanent,,Yes,Experienced with over 2 years experience,,Bachelors,Yes,,,,
3,Technician Resume,Engineering,83864,"Edmonton, Alberta",Part-Time Permanent,22$/hour,,New Grad/Entry Level,"Yes, 50-75%",Bachelors,Yes,,,,
4,Mechanical Engineer Resume,Engineering,83803,"Calgary, Alberta",Full-Time Permanent,,No,New Grad/Entry Level,,Bachelors,Undecided,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11824,management Resume,Business/Management,26166,"Arlington, Virginia",Full-Time Permanent,50000,Yes,"Management (Manager, Director)","Yes, Less Than 25%",High School/Equivalent,No,,,,
11825,Office Administration Resume,Business/Management,26160,"Queen Creek, Arizona",Full-Time Permanent,"41,000",Yes,Experienced with over 2 years experience,,Bachelors,No,,,,
11826,"CEO, The Associated Consultants, LLC Resume",Business/Management,26142,"Gaithersburg, Maryland",Full-Time Permanent,,,"Executive (President, VP, CEO)",,Masters,,,,,
11827,"Sales, Account Manager, Business Development, ...",Business/Management,26118,"Winchester, Virginia",Full-Time Permanent,"65,000.00",Yes,"Management (Manager, Director)",,Bachelors,No,,,,


In [24]:
from sklearn.model_selection import train_test_split
feature = df_copy['Desired Industry'].values
X = df_copy.drop('Desired Industry', axis=1)

In [ ]:
X_train